In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from typing import List, Tuple
import pickle

2025-06-12 23:36:20.051566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-12 23:36:20.158346: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 23:36:21.207208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import sys
sys.path.append('../../../../')

In [3]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-06-12 23:36:24.467770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
fold = folds_training_dicts[0]
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test', 'coordinates_train', 'coordinates_validation', 'coordinates_test'])

In [5]:
input_data = fold['components_train'][:100]
coordinates = fold['coordinates_train'][:100]
size_value = fold['sizes_train'][:100]
n_patches_hot_encoded_value = fold['num_patches_train'][:100]
max_number_of_patches = 10

In [7]:
from model import build_model

In [8]:
from models.patch_to_score.bootstrappers.model import bootstrap_patch_attention

In [12]:
model = bootstrap_patch_attention(features_mlp_hidden_sizes=[[256, 256]], features_mlp_dropout_rate=0.2, 
                    output_mlp_hidden_sizes=[[256, 256]], output_mlp_dropout_rate=0.2, 
                    attention_mlp_hidden_sizes=[[256, 256]], attention_mlp_dropout_rate=0.2,
                    activation='relu', input_shape=(10, 9), max_number_of_patches=10, attention_dimension=256, 
                    pairs_channel_dimension=256, num_heads=8, use_pair_bias=False)

initializing projection for skip connection: 11 -> 256


In [13]:
res = model([input_data, coordinates, size_value, n_patches_hot_encoded_value])
res

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[0.40484196],
       [0.4976446 ],
       [0.4491424 ],
       [0.49535605],
       [0.4491703 ],
       [0.4994908 ],
       [0.38405263],
       [0.4632912 ],
       [0.40825528],
       [0.48979607],
       [0.7339641 ],
       [0.45841724],
       [0.47305843],
       [0.44902006],
       [0.5817804 ],
       [0.5571623 ],
       [0.5041565 ],
       [0.64049214],
       [0.4346228 ],
       [0.58352435],
       [0.5309527 ],
       [0.4961424 ],
       [0.45643392],
       [0.50306857],
       [0.46662256],
       [0.40193903],
       [0.58649313],
       [0.48764238],
       [0.5759961 ],
       [0.54749966],
       [0.57799685],
       [0.46626487],
       [0.6239841 ],
       [0.6557391 ],
       [0.49086702],
       [0.4999464 ],
       [0.5499892 ],
       [0.5542364 ],
       [0.4569497 ],
       [0.49385163],
       [0.41683075],
       [0.60687697],
       [0.45057508],
       [0.58417344],
       [0.6053762 ],
     

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 number_of_patches_input (Input  [(None, 11)]        0           []                               
 Layer)                                                                                           
                                                                                                  
 tf.math.argmax (TFOpLambda)    (None,)              0           ['number_of_patches_input[0][0]']
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1)           0           ['tf.math.argmax[0][0]']         
 ingOpLambda)                                                                                     
                                                                                              

## playground

In [10]:
import numpy as np

In [35]:
attention_dim = 6
max_number_of_patches = 10
num_heads = 2
head_dim = attention_dim // num_heads

In [ ]:
# tmp of the form: (batches, patches, features)
# tmp is the result of applying W_{k/q/v} over F
batch_num = 15
tmp = np.array([[[f for f in range(attention_dim)] for p in range(max_number_of_patches)] for b in range(batch_num)])
tmp = tf.convert_to_tensor(tmp, dtype=tf.float32)
tmp.shape

TensorShape([15, 10, 6])

In [63]:
batch, num_patches, attention_dim = tmp.shape
res = tf.reshape(tmp, (batch, num_patches, num_heads, head_dim))
res = tf.transpose(res, perm=[0, 2, 1, 3])  # (batch, num_heads, num_patches, head_dim)

In [79]:
tmp_2 = tf.transpose(res, perm=[0, 2, 1, 3])
tmp_2 = tf.reshape(tmp_2, (batch, num_patches, attention_dim))
tmp_2

<tf.Tensor: shape=(15, 10, 6), dtype=float32, numpy=
array([[[0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.]],

       [[0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.]],

       [[0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 

In [61]:
tf.reshape(tmp, (-1, num_heads, tmp.shape[1], head_dim))[0][0][0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 2.], dtype=float32)>

In [10]:
from model import create_broadcasted_features, apply_mlps, mask_inputs, create_masked_inputs
from models.patch_to_score.models.modules.global_sum_pooling import GlobalSumPooling
from models.patch_to_score.models.modules.patch_attention_with_pair_bias import PatchAttentionWithPairBias
from models.patch_to_score.models.modules.transformer_encoder_mlp import TransformerEncoderMLP
from models.patch_to_score.models.modules.pairs_transition import PairsTransition

In [11]:
patch_attention_module = PatchAttentionWithPairBias(30, 2)

In [12]:
features_mlp_hidden_sizes = attention_mlp_hidden_sizes = output_mlp_hidden_sizes = [[256, 256]]
features_mlp_dropout_rate = attention_mlp_dropout_rate = output_mlp_dropout_rate = 0.2
activation = 'relu'
pairs_channel_dimension = attention_dimension = 256
num_heads = 8
 
# [[256, 256]], 0.2, 
# [[256, 256]], 0.2,
# 'relu', (10, 9), 10, 256, 256, 8

In [14]:
# input_data, coordinates, size_value, n_patches_hot_encoded_value = create_inputs(input_shape, max_number_of_patches)
features, pairwise_distances = create_masked_inputs(input_data, coordinates, size_value, n_patches_hot_encoded_value, max_number_of_patches)
F = apply_mlps(features, features_mlp_hidden_sizes,features_mlp_dropout_rate, activation)
pairs_transition = PairsTransition(pairs_channel_dimension)
D = pairs_transition(pairwise_distances)
attention_output = PatchAttentionWithPairBias(attention_dimension, num_heads)([F, D])
attention_mlp_output = apply_mlps(attention_output, attention_mlp_hidden_sizes, attention_mlp_dropout_rate, activation)
global_pooling_output = GlobalSumPooling(data_format='channels_last')(attention_mlp_output)
output_mlp_output = apply_mlps(global_pooling_output, output_mlp_hidden_sizes, output_mlp_dropout_rate, activation)
output = tf.keras.layers.Dense(1, activation='sigmoid')(output_mlp_output)

initializing projection for skip connection: 11 -> 256


In [35]:
output

<tf.Tensor: shape=(76632, 1), dtype=float32, numpy=
array([[0.5533056 ],
       [0.619216  ],
       [0.5120182 ],
       ...,
       [0.41534168],
       [0.60217166],
       [0.72498924]], dtype=float32)>